In [1]:
from azureml.core import Workspace
ws = Workspace.from_config()


Failure while loading azureml_run_type_providers. Failed to load entrypoint azureml.scriptrun = azureml.core.script_run:ScriptRun._from_run_dto with exception (azure-mgmt-authorization 2.0.0 (/Users/axelsirota/repos/deploying-managing-models-azure/.venv/lib/python3.7/site-packages), Requirement.parse('azure-mgmt-authorization<1.0.0,>=0.40.0')).


In [4]:
import urllib.request
from azureml.core.model import Model

# Download model
from pathlib import Path

my_file = Path("./model.onnx")
if not my_file.exists():
    urllib.request.urlretrieve("https://aka.ms/bidaf-9-model", "./model.onnx")


In [5]:

# Register model
model = Model.register(ws, model_name="bidaf_onnx", model_path="./model.onnx")

Registering model bidaf_onnx


In [6]:
from azureml.core import Environment
from azureml.core.model import InferenceConfig

env = Environment(name="AzureML-onnxruntime-1.6-ubuntu18.04-py37-cpu-inference")
dummy_inference_config = InferenceConfig(
    environment=env,
    source_directory="./source_dir",
    entry_script="./echo_score.py",
)


In [7]:
from azureml.core.webservice import LocalWebservice

deployment_config = LocalWebservice.deploy_configuration(port=6789)

In [ ]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    dummy_inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)
print(service.get_logs())

In [15]:
from azureml.core.model import InferenceConfig
from azureml.core.environment import CondaDependencies

env = Environment.from_pip_requirements(name="onnxruntime_env", file_path='./model_requirements.txt')
env.register(workspace=ws)


inference_config = InferenceConfig(environment=env, source_directory='./source_dir', entry_script='./score_real.py')

In [16]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Warning, azureml-defaults not detected in provided environment pip dependencies. The azureml-defaults package contains requirements for the inference stack to run, and should be included.
Warning, custom base image or base dockerfile detected without a specified `inferencing_stack_version`. Please set environment.inferencing_stack_version='latest'


Generating Docker build context.
2021/12/13 00:31:16 Downloading source code...
2021/12/13 00:31:17 Finished downloading source code
2021/12/13 00:31:17 Creating Docker network: acb_default_network, driver: 'bridge'
2021/12/13 00:31:18 Successfully set up Docker network: acb_default_network
2021/12/13 00:31:18 Setting up Docker configuration...
2021/12/13 00:31:18 Successfully set up Docker configuration
2021/12/13 00:31:18 Logging in to registry: testworkscrqba6t2dnjhh7y.azurecr.io
2021/12/13 00:31:19 Successfully logged into testworkscrqba6t2dnjhh7y.azurecr.io
2021/12/13 00:31:19 Executing step ID: acb_step_0. Timeout(sec): 5400, Working directory: '', Network: 'acb_default_network'
2021/12/13 00:31:19 Scanning for dependencies...
2021/12/13 00:31:19 Successfully scanned dependencies
2021/12/13 00:31:19 Launching container with name: acb_step_0
Sending build context to Docker daemon  66.56kB
Step 1/19 : FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20211029.v1@sha256:fda0c9

In [17]:
from azureml.core.webservice import AciWebservice

deployment_config = AciWebservice.deploy_configuration(
    cpu_cores=0.5, memory_gb=1, auth_enabled=True
)

In [18]:
service = Model.deploy(
    ws,
    "myservice",
    [model],
    inference_config,
    deployment_config,
    overwrite=True,
)
service.wait_for_deployment(show_output=True)

Tips: You can try get_logs(): https://aka.ms/debugimage#dockerlog or local deployment: https://aka.ms/debugimage#debug-locally to debug if deployment takes longer than 10 minutes.
Running
2021-12-12 21:43:38-03:00 Creating Container Registry if not exists.
2021-12-12 21:43:38-03:00 Registering the environment.
2021-12-12 21:43:39-03:00 Use the existing image.
2021-12-12 21:43:39-03:00 Generating deployment configuration.
2021-12-12 21:43:40-03:00 Submitting deployment to compute.
2021-12-12 21:43:43-03:00 Checking the status of deployment myservice..
2021-12-12 21:45:05-03:00 Checking the status of inference endpoint myservice.
Succeeded
ACI service creation operation finished, operation "Succeeded"


In [19]:
service.delete()
model.delete()